In [ ]:
# Import necessary libraries
import numpy as np 
import matplotlib.pyplot as plt
import os
import torch
import glob
from sklearn.preprocessing import LabelEncoder

# Import libraries for model training
import torch.nn as nn
from torch import optim
from torchvision import  transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.models import efficientnet_b0
from train import train_one_epoch
from evaluate import evaluate_model

# Import PIL for image processing
from PIL import Image

In [ ]:
# I used this Grad-CAM library to visualize the area that the model detected as meningioma tumor
# As the first step, I cloned the library from the github repository
!git clone https://github.com/jacobgil/pytorch-grad-cam.git
%cd pytorch-grad-cam
!pip install ttach

#Importing necessary libraries for Gradcam(These libraries need to clone a github library that I mentioned in readme file)
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image



In [ ]:
# I defined paths for training and testing data and used glob to get all the files in the directories
train_dir = "/kaggle/input/brain-tumor-mri-dataset/Training"
test_dir = "/kaggle/input/brain-tumor-mri-dataset/Testing"
train_files = glob.glob(train_dir + "/*/*")  
test_files = glob.glob(test_dir + "/*/*")
#here I wanted to make sure that I have same number of files in both training and testing directories
print(f"Number of training files: {len(train_files)}")
print(f"Number of testing files: {len(test_files)}")

In [ ]:
# I extracted the labels using the paths of the files then I used label encoder to encode the labels of the classes
train_labels =[]
for paths in train_files:
    label = paths.split('/')[5]
    train_labels.append(label)
    
test_labels =[]
for paths in test_files:
    label = paths.split('/')[5]
    test_labels.append(label)


label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
test_labels = label_encoder.transform(test_labels)

In [ ]:
# I defined a class for the dataset and used torchvision.transforms to resize the images to 224x224 and normalize them using ImageNet mean and std
class BrainTumorDataset(Dataset):
    def __init__(self, files_dir_list, labels, transforms=None):
        super().__init__()
        self.files_dir_list = files_dir_list
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index):
        image_path = self.files_dir_list[index]
        image = Image.open(image_path).convert("RGB")
        if self.transforms is not None:
            image = self.transforms(image)
        label = self.labels[index]
        return image,label
    
    def __len__(self):
        return(len(self.labels))



In [ ]:
my_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),

])

In [ ]:
# Here, I made datasets and dataloaders for training and testing data
train_dataset = BrainTumorDataset(train_files, train_labels, my_transforms)
test_dataset = BrainTumorDataset(test_files, test_labels, my_transforms)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# I used efficientnet_b0 model and changed the last layer to have 4 output features and used CrossEntropyLoss as loss function and Adam as optimizer
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = efficientnet_b0(pretrained=False)
model.classifier[1] = nn.Linear(in_features=1280, out_features=4, bias=True).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
# training the model for 20 epochs and saving the model's state_dict right after training
EPOCHS = 20

for epoch in range(EPOCHS):
    train_one_epoch(
        model,
        train_dataloader,
        optimizer,
        loss_fn,
        None,
        epoch,
        device=device
    )

torch.save(model.state_dict(), 'efficientnet_b0_on_nickparvar_20epochs_braintumor_dataset.pth')
print("Model state_dict saved.")

In [ ]:
#Then, I evaluated the model on the test data to get the accuracy and f1 score
evaluate_model(model, test_dataloader, device= torch.device("cuda" if torch.cuda.is_available() else "cpu"))


In [ ]:
# Finally, I uploaded a photo and used the model to predict the class of the photo, as it is obvious from the name of the file, it was a random meningiom tumor photo
# and the model predicted the class correctly
im_path = "/kaggle/input/mening/c72f1a6cc058e783ec8e518fbb0061_big_gallery.jpg"
image = Image.open(im_path).convert('RGB')
plt.imshow(image, cmap='gray')
image = my_transforms(image)

image = image.to(device)
model = model.to(device)  
output = model(image.unsqueeze(dim=0)).argmax()
final_output = output.cpu().numpy()

print(f'''All classes include:
    {label_encoder.classes_}''')
model_prediction = label_encoder.inverse_transform([final_output])
print(f'''Model's prediction for uploaded photo is{model_prediction}''')


In [ ]:
# For Grad-Cam implementation, I chosed 3rd layer of the model as the target layer
target_layers = [model.features[7][0]]
image_path = "/kaggle/input/mening/c72f1a6cc058e783ec8e518fbb0061_big_gallery.jpg"
input_image = Image.open(image_path).convert('RGB')
input_image = my_transforms(input_image).unsqueeze(0).to(device)

#meningioma is the target class since my photo is a meningioma tumor and I want to check if my model can detect the area
target_class = label_encoder.transform(["meningioma"])[0]  
targets = [ClassifierOutputTarget(target_class)]


with GradCAM(model=model, target_layers=target_layers) as cam:
    
    grayscale_cam = cam(input_tensor=input_image, targets=targets)
    grayscale_cam = grayscale_cam[0, :]  
    rgb_image = input_image[0].permute(1, 2, 0).cpu().numpy()  
    rgb_image = (rgb_image - rgb_image.min()) / (rgb_image.max() - rgb_image.min())  

    #The final photo consists of the heatmap of the area that the model detected as meningioma tumor as well as the original photo
    final_visualization = show_cam_on_image(rgb_image, grayscale_cam, use_rgb=True)


plt.imshow(final_visualization)
plt.title(f"Grad-CAM Heatmap for Predicted Class: Meningioma")
plt.axis("off")
plt.show()
